In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.5 MB/s eta 0:00:00


In [ ]:
import torch
import re
import pandas as pd
from collections import Counter
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Initialize model and tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained("/content/my_model")
    model = AutoModelForMaskedLM.from_pretrained("/content/my_model")
    print("Loaded fine-tuned model from /content/my_model")
except:
    print("Using pretrained model")
    model_name = "UBC-NLP/MARBERT"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)

Using pretrained model


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Move model to correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# **Data preprocessing functions**

In [ ]:
def preprocess(sentence: str) -> str:
    """Enhanced Arabic text preprocessing with additional normalization"""
    sentence = sentence.replace('أ', 'ا').replace('إ', 'ا').replace('آ', 'ا')
    sentence = re.sub(r'[^\u0600-\u06FF\s]', '', sentence)  # Keep only Arabic characters and spaces
    sentence = re.sub(r'\s+', ' ', sentence).strip()  # Clean up extra spaces
    return sentence


def data_vocab(dataframe, min_freq=3):
    """Create vocabulary with frequency filtering"""
    words_freq = Counter()
    for text in dataframe['text']:
        words_freq.update(text.split())
    return {word: freq for word, freq in words_freq.items() if freq >= min_freq}


def tokenize_function(examples):
    """Tokenization function for dataset"""
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

# **Prediction functions**

In [ ]:
def normalize_hamza(word: str) -> str:
    # Normalize common Hamza variants to unify them
    word = word.replace("أ", "ا").replace("إ", "ا").replace("ؤ", "و").replace("ئ", "ي").replace("ء", "")
    return word

def find_misspellings(text: str, vocab: dict, threshold: float = 0.28) -> list:
    """Identify potentially misspelled words using MLM probability and additional context"""
    words = text.split()
    misspelled_indices = []

    for i, word in enumerate(words):
        if word not in vocab and normalize_hamza(word) not in vocab:  
            masked_words = words.copy()
            masked_words[i] = tokenizer.mask_token
            masked_sentence = " ".join(masked_words)

            inputs = tokenizer(masked_sentence, return_tensors="pt").to(device)
            mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits[0, mask_token_index]
                probs = torch.softmax(logits, dim=-1).squeeze()
                word_id = tokenizer.encode(word, add_special_tokens=False)
                word_prob = torch.mean(probs[word_id]) if word_id else 0

            # Use lower threshold to catch more potential errors
            if word_prob < threshold:
                misspelled_indices.append(i)

    return misspelled_indices


def generate_masked_sentences(text: str, misspelled_indices: list) -> list:
    """Generate masked sentences for each misspelled word"""
    words = text.split()
    return [
        " ".join(words[:idx] + [tokenizer.mask_token] + words[idx + 1:])
        for idx in misspelled_indices
    ]


from Levenshtein import distance as levenshtein_distance

def predict(masked_sentence: str, top_k=25) -> list:
    """Predict top-k masked words from MLM"""
    inputs = tokenizer(masked_sentence, return_tensors="pt").to(device)
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits[0, mask_token_index]
    probs = torch.softmax(logits, dim=-1).squeeze()
    top_k_tokens = torch.topk(probs, top_k)

    predictions = []
    for token_id in top_k_tokens.indices:
        token = tokenizer.decode([token_id]).strip()
        if re.match(r'^[\u0600-\u06FF]{2,}$', token):
            predictions.append(token)

    return predictions

In [ ]:
test_cases = [
    ("وززارة [MASK] والتعليم", "التربية"),
    ("يوم [MASK]", "السبت")
]

for sentence, expected in test_cases:
    preds = predict(sentence)   
    is_correct = expected in preds
    print(f"Model {'✅' if is_correct else '❌'} {sentence} → {preds} (Expected: {expected})")


Model ✅ وززارة [MASK] والتعليم → ['التربية', 'التربيه', 'للتربية', 'الصحه', 'الصحة', 'والتربية', 'النقل', 'بالتربية', 'طيب', 'بالصحه'] (Expected: التربية)
Model ❌ يوم [MASK] → ['جميل', 'الجمعه', 'ميلادي', 'الخميس', 'مميز', 'الجمعة', 'حلو', 'لطيف', 'الاحد', 'سعيد'] (Expected: السبت)


# **Pipeline function**

In [ ]:
def pipeline(input_text: str, vocab: dict, verbose: bool = True) -> str:
    processed_text = preprocess(input_text)
    vocab = data_vocab(df, min_freq=3)
    misspelled_indices = find_misspellings(processed_text, vocab)

    if not misspelled_indices:
        if verbose:
            print("✅ لا توجد أخطاء إملائية واضحة.")
        return processed_text

    masked_sentences = generate_masked_sentences(processed_text, misspelled_indices)
    words = processed_text.split()
    corrections = {}

    for idx, masked in zip(misspelled_indices, masked_sentences):
        original_word = words[idx]
        candidates = predict(masked)
        if candidates:
            best_candidate = min(candidates, key=lambda c: levenshtein_distance(c, original_word))
            corrections[original_word] = best_candidate
            words[idx] = best_candidate

    corrected_sentence = " ".join(words)

    if verbose:
        print("🔍 الكلمات التي تم تصحيحها:")
        for original, corrected in corrections.items():
            print(f" - {original} ➤ {corrected}")

    return corrected_sentence

In [ ]:
# Load and prepare dataset
df = pd.read_csv('/content/drive/MyDrive/arabic_dataset_classifiction.csv')
print(df)

                                                     text  targe
0       بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...      0
1       قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...      0
2       أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...      0
3       اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...      0
4       تزال صناعة الجلود في المغرب تتبع الطريقة التقل...      0
...                                                   ...    ...
111723  اللاعب تأخر في العودة إلى التداريب والمدرب غاض...      4
111724  المشرف العام لحسنية أكادير قال إنه سيغادر الفر...      4
111725  نسب إليه نتائج الوداد وصحوة الرجاء وآخر صيحاته...      4
111726  ستحتضن الرباط في الفترة مابين يوليوز المقبل دو...      4
111727  رضخ أحمد أهمو رئيس أمل تيزنيت لكرة القدم لضغوط...      4

[111728 rows x 2 columns]


In [ ]:
# Clean dataset
df = df.drop(columns=['targe'], errors='ignore').dropna().drop_duplicates()
df['text'] = df['text'].apply(preprocess)
df['text'] = df['text'].apply(lambda x: x if len(x.split()) > 5 else None)
df = df.dropna().reset_index(drop=True)

In [ ]:
# Create vocabulary
words_freq = data_vocab(df)

In [ ]:
# Prepare Hugging Face dataset
dataset = Dataset.from_pandas(df[:10000])  # Use smaller subset for training
dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])
dataset = dataset.train_test_split(test_size=0.2, shuffle=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Training configuration
training_args = TrainingArguments(
    output_dir="./model",
    eval_strategy="epoch",
    learning_rate=3e-5,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    fp16=True,
    save_strategy="epoch",
    logging_steps=50,
    optim="adafactor"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-18-d34e912acbdb>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train and save model
trainer.train()
trainer.save_model("/content/my_model")
tokenizer.save_pretrained("/content/my_model")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: habdelhamed729 (habdelhamed729-institution-of-engineering-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.529800,3.141116
2,3.131800,3.045348
3,3.034000,2.963398
4,2.822700,2.896676
5,2.762800,2.832915
6,2.732800,2.773669
7,2.625000,2.763121
8,2.579400,2.740458
9,2.541500,2.723837
10,2.499100,2.738370


('/content/my_model/tokenizer_config.json',
 '/content/my_model/special_tokens_map.json',
 '/content/my_model/vocab.txt',
 '/content/my_model/added_tokens.json',
 '/content/my_model/tokenizer.json')

In [ ]:
input_texts = [
    "وززارة النربية والتعليم تعلن عن تعطبل الدراسة رسميا يوم السسبت نظرا للظروف الجوية الحالييه وحفاظا على سلامة المعلمون و الططلاب",
    "بعد أن قرر القضاء الفنسي الإبقاء على المغني المغربي سعد لمجرد الذي قبض عليه يوم الحميس بتهمتي القتتل و التحرش",
    "قال الفنان المغربي محمد الخياري نحن احرار الامه الاسلاميه و لن نسمح بحدوث حربب في الوطن",
    "الطلاب يذههبون الى المدرسه في الصباح الباككر لتلقي الدروس",
    "وصل الريس الى القاهر صباح اليوم لعقد اجتماع مهم مع الوزراء",
    "يعاني المزارعون من مشاكل في توفر المايه للري في فصل الصيف",
    "اصدر القاضي حكمه النهائي بعد مداوله طويله بين اعضاء المحكمه",
    "المنتخب الوطني يخض مباراة مهمه في التصفيات المؤهله لكأس العاللم",
]

for input_text in input_texts:
    # Pass words_freq as the vocab argument
    true_sentence = pipeline(input_text, words_freq)
    print('Incorrect Sentence:', input_text)
    print('Corrected Sentence:', true_sentence)
    print('-' * 20)

🔍 الكلمات التي تم تصحيحها:
 - وززارة ➤ وزارة
 - النربية ➤ التربية
 - تعطبل ➤ تعطيل
 - السسبت ➤ السبت
 - الحالييه ➤ الحالية
 - المعلمون ➤ المعلمين
 - الططلاب ➤ الطلاب
Incorrect Sentence: وززارة النربية والتعليم تعلن عن تعطبل الدراسة رسميا يوم السسبت نظرا للظروف الجوية الحالييه وحفاظا على سلامة المعلمون و الططلاب
Corrected Sentence: وزارة التربية والتعليم تعلن عن تعطيل الدراسة رسميا يوم السبت نظرا للظروف الجوية الحالية وحفاظا على سلامة المعلمين و الطلاب
--------------------
🔍 الكلمات التي تم تصحيحها:
 - الفنسي ➤ الفرنسي
 - الحميس ➤ الخميس
 - القتتل ➤ القتل
Incorrect Sentence: بعد أن قرر القضاء الفنسي الإبقاء على المغني المغربي سعد لمجرد الذي قبض عليه يوم الحميس بتهمتي القتتل و التحرش
Corrected Sentence: بعد ان قرر القضاء الفرنسي الابقاء على المغني المغربي سعد لمجرد الذي قبض عليه يوم الخميس بتهمتي القتل و التحرش
--------------------
🔍 الكلمات التي تم تصحيحها:
 - الاسلاميه ➤ الاسلامية
 - حربب ➤ حرب
Incorrect Sentence: قال الفنان المغربي محمد الخياري نحن احرار الامه الاسلاميه و لن نسمح بحدو

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np

# Define test dataset with ground-truth corrections
test_df = pd.DataFrame({
    'incorrect': input_texts,
    'correct': [
        "وزارة التربية والتعليم تعلن عن تعطيل الدراسة رسميا يوم السبت نظرا للظروف الجوية الحالية وحفاظا على سلامة المعلمون و الطلاب",
        "بعد ان قرر القضاء الفرنسي الابقاء على المغني المغربي سعد لمجرد الذي قبض عليه يوم الخميس بتهمتي القتل و التحرش",
        "قال الفنان المغربي محمد الخياري نحن احرار الامه الاسلامية و لن نسمح بحدوث حرب في الوطن",
        "الطلاب يذهبون الى المدرسة في الصباح الباكر لتلقي الدروس",
        "وصل الريس الى القاهرة صباح اليوم لعقد اجتماع مهم مع الوزراء",
        "يعاني المزارعون من مشاكل في توفر الماء للري في فصل الصيف",
        "اصدر القاضي حكمه النهائي بعد مداولة طويلة بين اعضاء المحكمة",
        "المنتخب الوطني يخوض مباراة مهمة في التصفيات المؤهلة لكاس العالم"
    ]
})

def evaluate_pipeline(test_df, pipeline_func, df):
    y_true_words = []
    y_pred_words = []
    sentence_correct = []

    for idx, row in test_df.iterrows():
        incorrect = row['incorrect']
        true_sentence = row['correct']

        # Run pipeline
        pred_sentence = pipeline_func(incorrect, df, verbose=False)

        # Split sentences into words
        true_words = true_sentence.split()
        pred_words = pred_sentence.split()
        incorrect_words = incorrect.split()

        # Ensure same length for comparison
        min_len = min(len(true_words), len(pred_words), len(incorrect_words))
        true_words = true_words[:min_len]
        pred_words = pred_words[:min_len]
        incorrect_words = incorrect_words[:min_len]

        # Word-level comparison for misspelled words
        for i in range(min_len):
            if incorrect_words[i] != true_words[i]:  # Misspelled word
                y_true_words.append(true_words[i])
                y_pred_words.append(pred_words[i])

        # Sentence-level accuracy
        sentence_correct.append(true_sentence == pred_sentence)

    # Calculate metrics
    word_accuracy = accuracy_score(y_true_words, y_pred_words)
    word_f1 = f1_score(y_true_words, y_pred_words, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_true_words, y_pred_words)
    sentence_accuracy = np.mean(sentence_correct)

    return {
        'word_accuracy': word_accuracy,
        'word_f1': word_f1,
        'confusion_matrix': conf_matrix,
        'sentence_accuracy': sentence_accuracy,
        'y_true_words': y_true_words,
        'y_pred_words': y_pred_words
    }

In [ ]:
# Evaluation Section
print("\n=== Evaluation Section ===")

# Run evaluation
results = evaluate_pipeline(test_df, pipeline, df)

# Print evaluation results
print("Evaluation Results:")
print(f"Word-Level Accuracy: {results['word_accuracy']:.4f}")
print(f"Word-Level F1 Score: {results['word_f1']:.4f}")
print(f"Sentence-Level Accuracy: {results['sentence_accuracy']:.4f}")
print("Confusion Matrix (Word-Level):")
print(results['confusion_matrix'])
print("\nDetailed Errors:")
for true, pred in zip(results['y_true_words'], results['y_pred_words']):
    if true != pred:
        print(f"True: {true}, Predicted: {pred}")


=== Evaluation Section ===
Evaluation Results:
Word-Level Accuracy: 0.8462
Word-Level F1 Score: 0.8462
Sentence-Level Accuracy: 0.6250
Confusion Matrix (Word-Level):
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 

In [ ]:
from google.colab import files

!zip -r my_model.zip /content/my_model
files.download('my_model.zip')

updating: content/my_model/ (stored 0%)
updating: content/my_model/vocab.txt (deflated 61%)
updating: content/my_model/tokenizer_config.json (deflated 74%)
updating: content/my_model/config.json (deflated 53%)
updating: content/my_model/special_tokens_map.json (deflated 42%)
updating: content/my_model/model.safetensors (deflated 7%)
updating: content/my_model/tokenizer.json (deflated 72%)
updating: content/my_model/training_args.bin (deflated 52%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>